# Лабораторная 2. Формирование отчётов в Apache Spark

## Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. 
Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

In [1]:
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql import SparkSession

import os
import sys

In [2]:
# Задаем переменные окружения для корректной работы некоторых функций
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Задаем переменную окружения для парсинга xml
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.17.0 pyspark-shell'

os.chdir('../')

In [3]:
spark = SparkSession.builder.getOrCreate()
spark

In [4]:
def show_info(data):
    print("Схема:")
    data.printSchema()
    
    print("Первые 5 элементов:")
    data.show(n = 5)
    
    print("Описание датасета:")
    data.describe().show()
    
    print("Количество элементов:")
    print(data.count())

In [5]:
def get_path_to_file(file_name):
    return f'file:///{os.getcwd()}/{file_name}'.replace("\\", "/")

In [6]:
postsData = spark.read.format('xml').option('rowTag', 'row').option("timestampFormat", 'y/M/d H:m:s').load(get_path_to_file('data/posts_sample.xml'))
show_info(postsData)

Схема:
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)

Первые 5 элементов:
+-----------------+------------+------------------

In [8]:
# Ищем с 1 января 2010 года до 31 декабря 2020 года
dates = ("2010-01-01",  "2020-12-31")
posts_by_date = postsData.filter(F.col("_CreationDate").between(*dates))
posts_by_date.show(10)

+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|_Tags|_Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|             NULL|        NULL|<p>No. (And more ...|       NULL

In [9]:
languagesData = spark.read.format('csv').option('header', 'true').option("inferSchema", True).load(get_path_to_file('data/programming-languages.csv')).dropna()
show_info(languagesData)

Схема:
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)

Первые 5 элементов:
+----------+--------------------+
|      name|       wikipedia_url|
+----------+--------------------+
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
|A-0 System|https://en.wikipe...|
|        A+|https://en.wikipe...|
|       A++|https://en.wikipe...|
+----------+--------------------+
only showing top 5 rows

Описание датасета:
+-------+--------+--------------------+
|summary|    name|       wikipedia_url|
+-------+--------+--------------------+
|  count|     699|                 699|
|   mean|    NULL|                NULL|
| stddev|    NULL|                NULL|
|    min|@Formula|https://en.wikipe...|
|    max|xHarbour|https://en.wikipe...|
+-------+--------+--------------------+

Количество элементов:
699


In [10]:
language_names = [str(x[0]) for x in languagesData.collect()]
language_names

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS',
 'ACC',
 'Accent',
 'Ace DASL',
 'ACL2',
 'ACT-III',
 'Action!',
 'ActionScript',
 'Ada',
 'Adenine',
 'Agda',
 'Agilent VEE',
 'Agora',
 'AIMMS',
 'Alef',
 'ALF',
 'ALGOL 58',
 'ALGOL 60',
 'ALGOL 68',
 'ALGOL W',
 'Alice',
 'Alma-0',
 'AmbientTalk',
 'Amiga E',
 'AMOS',
 'AMPL',
 'Apex (Salesforce.com)',
 'APL',
 "App Inventor for Android's visual block language",
 'AppleScript',
 'Arc',
 'ARexx',
 'Argus',
 'AspectJ',
 'Assembly language',
 'ATS',
 'Ateji PX',
 'AutoHotkey',
 'Autocoder',
 'AutoIt',
 'AutoLISP / Visual LISP',
 'Averest',
 'AWK',
 'Axum',
 'B',
 'Babbage',
 'Bash',
 'BASIC',
 'bc',
 'BCPL',
 'BeanShell',
 'Batch (Windows/Dos)',
 'Bertrand',
 'BETA',
 'Bigwig',
 'Bistro',
 'BitC',
 'BLISS',
 'Blockly',
 'BlooP',
 'Blue',
 'Boo',
 'Boomerang',
 'Bourne shell (including',
 'bash and',
 'ksh )',
 'BREW',
 'BPEL',
 'C',
 'C--',
 'C++ – ISO/IEC 14882',
 'C# – ISO/IEC

In [11]:
# Функция позволяет определить какой язык содержится в теге поста
def includes_name(x):
    tag = None
    for name in language_names:
        n = '<' + name.lower() + '>'
        if n in str(x._Tags).lower():
            tag = name
            break
    if tag is None:
        tag = 'No'
            
    return (x[6], tag)

In [13]:
# Получаем посты именно с языками программирования
posts_by_date_rdd = posts_by_date.rdd.map(includes_name).filter(lambda x: x[1] != 'No')
posts_by_date_rdd_group = posts_by_date_rdd.keyBy(lambda row: (row[0].year, row[1])).aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2).sortBy(lambda x: x[1], ascending=False).collect()

In [14]:
years_list = [i for i in range(2010, 2021)][::-1]
df_by_years = []
for year in years_list:
    df_by_years.extend([row for row in posts_by_date_rdd_group if row[0][0] == year][:10])

In [15]:
row_template = Row('Year', 'Language', 'Count')
result_df = spark.createDataFrame([row_template(*x, y) for x, y in df_by_years])

In [16]:
result_df.show()

+----+----------+-----+
|Year|  Language|Count|
+----+----------+-----+
|2019|    Python|  162|
|2019|JavaScript|  131|
|2019|      Java|   95|
|2019|       PHP|   59|
|2019|         R|   36|
|2019|         C|   14|
|2019|        Go|    9|
|2019|      Dart|    9|
|2019|    MATLAB|    9|
|2019|    Kotlin|    9|
|2018|    Python|  214|
|2018|JavaScript|  196|
|2018|      Java|  145|
|2018|       PHP|   99|
|2018|         R|   63|
|2018|         C|   24|
|2018|     Scala|   22|
|2018|TypeScript|   21|
|2018|PowerShell|   13|
|2018|      Bash|   12|
+----+----------+-----+
only showing top 20 rows



## 2. Получившийся отчёт сохранить в формате Apache Parquet.

In [17]:
os.chdir('L2 - Reports with Apache Spark')

In [18]:
result_df.write.mode("overwrite").parquet(get_path_to_file("top_ten_languages.parquet"))